In [1]:
import pandas as pd
import pickle

/home/mitya/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
with open('data/sentiment_dataset/mentions texts.pickle', 'rb') as file:
    mentions_texts = pickle.load(file)

with open('data/sentiment_dataset/sentiment_texts.pickle', 'rb') as file:
    sent_texts = pickle.load(file)
    
mentions = pd.read_csv('data/sentiment_dataset/mentions.csv', index_col=0)
sent = pd.read_csv('data/sentiment_dataset/sentiment.csv', index_col=0)

In [3]:
issuers = pd.read_excel('data/sentiment_dataset/issuers.xlsx', index_col=0)
synonyms = pd.read_excel('data/sentiment_dataset/names and synonyms.xlsx')

In [4]:
issuers.head()

,issuerid,EMITENT_FULL_NAME,datetrackstart,datetrackend,BGTicker,OtherTicker
0,1,"""Акционерный коммерческий банк ""Держава"" публи...",2021-06-02 12:47:55.100,NaN,NaN,NaN
1,2,"""МОСКОВСКИЙ КРЕДИТНЫЙ БАНК"" (публичное акционе...",2021-06-02 12:47:55.100,NaN,CBOM RX,NaN
2,3,"""Российский акционерный коммерческий дорожный ...",2021-06-02 12:47:55.100,NaN,NaN,NaN
3,4,"Акционерная компания ""АЛРОСА"" (публичное акцио...",2021-06-02 12:47:55.100,NaN,ALRS RX,NaN
4,5,"Акционерный Коммерческий банк ""АВАНГАРД"" - пуб...",2021-06-02 12:47:55.100,NaN,NaN,NaN


In [5]:
synonyms.head()

,issuerid,EMITENT_FULL_NAME,VeryOddCompany,BGTicker,BGTicker.1,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,1,"Акционерный коммерческий банк ""Держава"" публич...",NaN,NaN,DERZP,Держава,DERZHAVA,DERZ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,"""МОСКОВСКИЙ КРЕДИТНЫЙ БАНК"" (публичное акционе...",NaN,CBOM RX,NaN,Московский кредитный банк,мкб,Credit Bank of Moscow,Credit Bank,NaN,NaN,NaN,NaN,NaN,NaN
2,3,"""Российский акционерный коммерческий дорожный ...",NaN,NaN,RDRB,Российский акционерный коммерческий дорожный банк,РДБанк,Дорожный банк,Russian public joint-stock commercial roads Bank,RosDorBank,РосДорБанк,roads Bank,NaN,NaN,NaN
3,4,"Акционерная компания ""АЛРОСА"" (публичное акцио...",NaN,ALRS RX,NaN,алроса,alrosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,"Акционерный Коммерческий банк ""АВАНГАРД"" - пуб...",NaN,NaN,AVAN,Авангард,AVANGARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
issuers.issuerid.nunique()

255

In [7]:
import re

synonyms_dict = {}
for i in range(synonyms.shape[0]):

    idx = synonyms.issuerid.iloc[i]
    synonyms_dict[idx] = synonyms.iloc[i].dropna().values[1:]
    synonyms_dict[idx] = [x.strip().lower() for x in synonyms_dict[idx]]

    # Remove rx followed by space or with space before ("rx " or " rx")
    synonyms_dict[idx] = [re.sub(r'rx\s', '', x) for x in synonyms_dict[idx]]
    synonyms_dict[idx] = [re.sub(r'\srx', '', x) for x in synonyms_dict[idx]]
    
synonyms_dict

{1: ['акционерный коммерческий банк "держава" публичное акционерное общество',
  'derzp',
  'держава',
  'derzhava',
  'derz'],
 2: ['"московский кредитный банк" (публичное акционерное общество)',
  'cbom',
  'московский кредитный банк',
  'мкб',
  'credit bank of moscow',
  'credit bank'],
 3: ['"российский акционерный коммерческий дорожный банк" (публичное акционерное общество)',
  'rdrb',
  'российский акционерный коммерческий дорожный банк',
  'рдбанк',
  'дорожный банк',
  'russian public joint-stock commercial roads bank',
  'rosdorbank',
  'росдорбанк',
  'roads bank'],
 4: ['акционерная компания "алроса" (публичное акционерное общество)',
  'alrs',
  'алроса',
  'alrosa'],
 5: ['акционерный коммерческий банк "авангард" - публичное акционерное общество',
  'avan',
  'авангард',
  'avangard'],
 6: ['акционерный коммерческий банк "приморье"  (публичное акционерное общество)',
  'prmb',
  'акб приморье',
  'акб "приморье"',
  'банк "приморье"',
  'банк приморье',
  'primbank',
  'p

In [8]:
# Merge ChannelID and messageid into single key
mentions_texts['MessageID'] = mentions_texts.ChannelID.astype(str) + mentions_texts.messageid.astype(str)

mentions_texts.drop(columns=['ChannelID', 'messageid'], inplace=True)

In [9]:
mentions_texts

,issuerid,MessageID,DateAdded,DatePosted,MessageText,IsForward
0,90,11972104335408,2021-02-06 01:42:42,2020-04-29 07:29:01,?? Фокус недели #ФН Сегодня ????? ММК опублик...,False
1,57,120356056764803,2021-02-06 01:47:00,2020-01-21 12:51:42,??#LSRG ЛСР - операционные результаты (2019г)...,False
2,152,119721043323389,2021-07-21 13:46:31,2021-07-21 11:15:46,#CHMF Северсталь (CHMF) впервые поставила в Бр...,False
3,112,1066174394677,2021-09-21 04:23:59,2016-12-16 10:00:04,"""Версия: Многоходовочка по Роснефти Роснефтег...",False
4,115,12394059894486,2023-01-20 15:02:22,2023-01-20 13:03:38,🟢 Новости к этому часу ⚪️ФРС США необходимо ...,False
...,...,...,...,...,...,...
19350,116,13266568111758,2021-02-06 19:05:52,2019-07-10 20:01:05,"""#NLMK """"Новолипецкий металлургический комбина...",False
19351,218,10751012066196,2021-02-06 01:02:00,2020-10-15 10:48:10,?? За 2020 год на МосБирже может появиться 10 ...,False
19352,227,1203560567141069,2021-06-15 14:30:24,2021-06-04 08:01:55,"""????#FIVE #ритейлеры #россия СДЕЛКИ """"МАГНИТ...",False
19353,115,1160971567283,2021-09-21 04:33:17,2020-02-17 23:11:07,UPD: ТА Интер РАО UPD: ТА Новатэк,False


In [10]:
all_mentions = mentions_texts.groupby('MessageID').issuerid.apply(list)
all_mentions.head(5)

MessageID
10010295601113      [32]
10010295601177      [62]
100102956012638    [235]
100102956012745    [235]
100102956012818    [235]
Name: issuerid, dtype: object

In [11]:
lens = all_mentions.apply(len)
lens.value_counts()

issuerid
1     15118
2      1073
3       276
4       105
5        56
6        21
8        13
7        10
9         6
11        6
10        2
13        1
17        1
40        1
37        1
16        1
Name: count, dtype: int64

In [12]:
texts = mentions_texts[['MessageID', 'MessageText']].drop_duplicates()
texts.head()

,MessageID,MessageText
0,11972104335408,?? Фокус недели #ФН Сегодня ????? ММК опублик...
1,120356056764803,??#LSRG ЛСР - операционные результаты (2019г)...
2,119721043323389,#CHMF Северсталь (CHMF) впервые поставила в Бр...
3,1066174394677,"""Версия: Многоходовочка по Роснефти Роснефтег..."
4,12394059894486,🟢 Новости к этому часу ⚪️ФРС США необходимо ...


In [13]:
from tqdm import tqdm
import re

problematic_cnt = 0
problematic_ids = []
found_keywords = {}

iou_sum = 0

for row in tqdm(texts.itertuples(), total=texts.shape[0]):
    msg_id = row.MessageID
    text = row.MessageText.lower()
    
    if msg_id in all_mentions.index:
        issuer_ids = all_mentions[msg_id]
        found_ids = []
        for id, syns in synonyms_dict.items():
            for syn in syns:
                if syn in text:
                    found_ids.append(id)
                    if msg_id in found_keywords:
                        found_keywords[msg_id].append(syn)
                    else:
                        found_keywords[msg_id] = [syn]
                    break
        
        iou = len(set(issuer_ids).intersection(found_ids)) / len(set(issuer_ids).union(found_ids))
        iou_sum += iou
        if iou < 0.5:
            problematic_cnt += 1
            problematic_ids.append(msg_id)
    else:
        print('No mentions')
        print(msg_id)
        raise ValueError
    

100%|██████████| 16691/16691 [00:07<00:00, 2274.58it/s]


In [14]:
print('Problematic percentage:', problematic_cnt / texts.shape[0])
print('Mean IOU:', iou_sum / texts.shape[0])

Problematic percentage: 0.3409621951950153
Mean IOU: 0.608487313707751


In [15]:
mentions_texts[mentions_texts.messageid == 5818]

AttributeError: 'DataFrame' object has no attribute 'messageid'

In [16]:
for id in problematic_ids[-10:]:
    print(id)
    print(texts.loc[texts.MessageID == id].MessageText.values)
    print('Found:', found_keywords.get(id))
    
    real_mentions = []
    for issuer_id in all_mentions[id]:
        real_mentions.append(synonyms_dict[issuer_id])
    print('Real:', real_mentions)
        
    
    print()

119721043320049
['?? Что было сегодня?  ?? Опубликована финансовая отчетность Мосбиржи за 1 кв. 2021#MOEX #Отчетность   ?? Мосбиржа планирует до конца 2021г запустить утренние торги на рынке акций с 7:00 мск #MOEX   ?? Минэнерго предлагает установить запрет на экспорт бензина из РФ сроком на три месяца #Нефтянка   ??  Опубликована финансовая отчетность КуйбышевАзот за 1 кв. 2021 по РСБУ #KAZT #Отчетность   ??Опубликован индекс потребительских цен в Еврозоне в апреле (г/г) #Макро #Статистика  ??  Опубликована финансовая отчетность Аэрофлота за 1 кв. 2021 по РСБУ #AFLT #Отчетность   ?? Турция разрешила использование Спутник V #Вакцина   ?? СД Газпрома обсудит дивиденды 20 мая #GAZP #Дивиденды']
Found: ['aflt', 'gazp', 'kazt', 'moex']
Real: [['публичное акционерное общество "газпром"', 'gazp', 'пао "газпром"', 'газпром', 'gazprom', 'gazp']]

112517223240212
['Кто «кормит» Севастополь? По оперативным данным за 2021 год в доход бюджета города Севастополя поступило 63 512,0 млн рублей, в том

In [17]:
import importlib
import final_solution
importlib.reload(final_solution)

from final_solution import baseline
importlib.reload(baseline)

<module 'final_solution.baseline' from '/home/mitya/Documents/gagarin_hack/hakaton-gagarin-sentiment_interface/final_solution/baseline.py'>

In [18]:
final_solution.baseline.find_entities(['Новатэк амогус тинькофф сбер', 'сбер он и в африке сбер'])

100%|██████████| 2/2 [00:00<00:00, 646.97it/s]


[[(115, (0, 7)), (150, (24, 28)), (225, (15, 23))], [(150, (0, 4))]]

In [19]:
found_entities = final_solution.baseline.find_entities(texts.MessageText.values)

100%|██████████| 16691/16691 [01:17<00:00, 215.79it/s]


In [25]:
found_entities[0]

[(53, (905, 909)),
 (62, (653, 657)),
 (89, (168, 172)),
 (90, (87, 91)),
 (103, (1380, 1384)),
 (115, (248, 252)),
 (116, (1061, 1065)),
 (185, (994, 998)),
 (189, (1637, 1641)),
 (197, (740, 744)),
 (223, (316, 318))]

In [22]:
real_entities = [all_mentions.get(id, []) for id in texts.MessageID.values]

# Compute macro-averaged F1 for all classes (ids)

f1_scores = []
for real, found in zip(real_entities, found_entities):
    real = set(real)
    found = set(tp[0] for tp in found)
    
    tp = len(real.intersection(found))
    fp = len(found - real)
    fn = len(real - found)
    
    if tp == 0:
        f1 = 0
    else:
        precision = tp / (tp + fp)
        recall = tp / (tp + fn)
        f1 = 2 * precision * recall / (precision + recall)
    
    f1_scores.append(f1)
    
f1_scores = [x for x in f1_scores if not pd.isna(x)]
print('Mean F1:', sum(f1_scores) / len(f1_scores))

Mean F1: 0.6864440583123663


Почему сентимент разный????

In [61]:
sent_texts[sent_texts['MessageID'] == 235445].iloc[0]['MessageText']

'"🛢🇷🇺🇿🇦#TRNFP ""Транснефть"" выразила заинтересованность в сотрудничестве с Sasol и Transnet из ЮАР"'

In [62]:
sent_texts[sent_texts['MessageID'] == 235445]

,MessageID,ChannelID,issuerid,SentimentScore,DateAdded,DatePosted,MessageText,IsForward
54,235445,1203560567,175,3,2023-03-30 14:50:10,2023-03-30 13:41:08,"""🛢🇷🇺🇿🇦#TRNFP """"Транснефть"""" выразила заинтерес...",False
55,235445,1203560567,175,4,2023-03-30 14:50:10,2023-03-30 13:41:08,"""🛢🇷🇺🇿🇦#TRNFP """"Транснефть"""" выразила заинтерес...",False


In [63]:
sent_texts_keys = sent_texts.set_index(['MessageID', 'ChannelID', 'issuerid'])
sent_keys = sent.set_index(['MessageID', 'ChannelID', 'issuerid'])
merged = sent_texts_keys.merge(sent_keys, on=['MessageID', 'ChannelID', 'issuerid'], suffixes=('_left', '_right'))
merged[merged['SentimentScore_left'] != merged['SentimentScore_right']]

SentimentScore_left           DateAdded  \
MessageID ChannelID  issuerid                                            
235445    1203560567 175                         3 2023-03-30 14:50:10   
                     175                         4 2023-03-30 14:50:10   
1485      1565800335 127                         5 2023-04-26 19:20:22   
                     127                         4 2023-04-26 19:20:22   
                     90                          2 2023-04-26 19:20:22   
...                                            ...                 ...   
3681      1219484576 127                         3 2023-03-16 20:59:15   
236494    1203560567 99                          4 2023-04-06 14:52:12   
                     99                          3 2023-04-06 14:52:12   
15560     1001029560 111                         4 2023-10-27 19:14:11   
                     111                         3 2023-10-27 19:14:11   

                                       DatePosted  \
MessageID ChannelID  issuerid                       
235445    1203560567 175      2023-03-30 13:41:08   
                     175      2023-03-30 13:41:08   
1485      1565800335 127      2023-04-26 14:22:10   
                     127      2023-04-26 14:22:10   
                     90       2023-04-26 14:22:10   
...                                           ...   
3681      1219484576 127      2023-03-16 15:27:53   
236494    1203560567 99       2023-04-06 13:39:49   
                     99       2023-04-06 13:39:49   
15560     1001029560 111      2023-10-27 16:27:42   
                     111      2023-10-27 16:27:42   

                                                                     MessageText  \
MessageID ChannelID  issuerid                                                      
235445    1203560567 175       "🛢🇷🇺🇿🇦#TRNFP ""Транснефть"" выразила заинтерес...   
                     175       "🛢🇷🇺🇿🇦#TRNFP ""Транснефть"" выразила заинтерес...   
1485      1565800335 127       💡 Подборка лучших акций российских компаний дл...   
                     127       💡 Подборка лучших акций российских компаний дл...   
                     90        💡 Подборка лучших акций российских компаний дл...   
...                                                                          ...   
3681      1219484576 127       Изучаем финансовый отчет «Полюса» в 2022 году ...   
236494    1203560567 99        "🇷🇺#MTLR #суд Суд оставил без движения иск Газ...   
                     99        "🇷🇺#MTLR #суд Суд оставил без движения иск Газ...   
15560     1001029560 111       Топ обсуждаемых бумаг в «Профите» за неделю  1...   
                     111       Топ обсуждаемых бумаг в «Профите» за неделю  1...   

                               IsForward  SentimentScore_right  
MessageID ChannelID  issuerid                                   
235445    1203560567 175           False                     4  
                     175           False                     3  
1485      1565800335 127           False                     4  
                     127           False                     5  
                     90            False                     4  
...                                  ...                   ...  
3681      1219484576 127           False                     2  
236494    1203560567 99            False                     3  
                     99            False                     4  
15560     1001029560 111           False                     3  
                     111           False                     4  

[108 rows x 6 columns]

In [66]:
texts

,MessageID,MessageText
0,11972104335408,?? Фокус недели #ФН Сегодня ????? ММК опублик...
1,120356056764803,??#LSRG ЛСР - операционные результаты (2019г)...
2,119721043323389,#CHMF Северсталь (CHMF) впервые поставила в Бр...
3,1066174394677,"""Версия: Многоходовочка по Роснефти Роснефтег..."
4,12394059894486,🟢 Новости к этому часу ⚪️ФРС США необходимо ...
...,...,...
19350,13266568111758,"""#NLMK """"Новолипецкий металлургический комбина..."
19351,10751012066196,?? За 2020 год на МосБирже может появиться 10 ...
19352,1203560567141069,"""????#FIVE #ритейлеры #россия СДЕЛКИ """"МАГНИТ..."
19353,1160971567283,UPD: ТА Интер РАО UPD: ТА Новатэк


In [74]:
list_for_df_ans = []

for idx, (real, found) in tqdm(enumerate(zip(real_entities, found_entities)), total=len(real_entities)):
    real = set(real)
    similar = real.intersection(set(tp[0] for tp in found))
    for item_found in found:
        if item_found[0] not in similar:
            continue
        now_text = texts.iloc[idx]['MessageText']
        if len(sent_texts[sent_texts['MessageText'] == now_text]) == 0:
            continue
        score = sent_texts[sent_texts['MessageText'] == now_text]['SentimentScore'].iloc[0].item()
        list_for_df_ans.append((now_text, item_found[0], item_found[1][0], item_found[1][1], score))

100%|██████████| 16691/16691 [00:23<00:00, 722.36it/s]


In [77]:
df_ans = pd.DataFrame(list_for_df_ans, columns=['text', 'entity_idx', 'start_pos', 'end_pos', 'score'])
df_ans

,text,entity_idx,start_pos,end_pos,score
0,🟢 Новости к этому часу ⚪️ФРС США необходимо ...,115,1140,1144,3
1,"""🇷🇺#PLZL #отчетность 29 августа - """"Полюс"""" ...",127,4,8,3
2,"⛔️🇷🇺#YNDX #делистинг «Яндекс» сообщил, что бир...",236,5,9,2
3,Главное к открытию среды (20.09): #брифинг 📌 ...,209,476,480,2
4,AGRO в моменте 🚀,231,0,4,3
...,...,...,...,...,...
8920,Дивидендный гэп в Татнефти. Как быстро будет з...,163,507,511,3
8921,"""🇷🇺#TCSG """"Тинькофф"""" запустит онлайн-продажу...",225,4,8,4
8922,💥🇷🇺#SNGSp закрыл гэп июля 2022г,160,4,8,3
8923,SMLT +2% Недвига срет сегодня охуенно 🚀,56,0,4,3


In [78]:
df_ans.to_csv('data/sentiment_dataset/ans_v1.csv', index=False)